In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
image_height = 64
image_width = 64

datagen = ImageDataGenerator(
    rescale=1./255,        
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    zoom_range=0.05,         
    horizontal_flip=False,
    vertical_flip=False,
)

train_generator = datagen.flow_from_directory(
    'data2/train',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale'
)

validation_generator = datagen.flow_from_directory(
    'data2/validation',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale'
)

Found 603510 images belonging to 39 classes.
Found 146562 images belonging to 39 classes.


In [2]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential(name='model_eng')

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
    
model.add(Flatten())

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(train_generator.num_classes, activation='softmax'))

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)
    
model.summary()

Model: "model_eng"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 31, 31, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 64)        0 

In [3]:
import warnings
from tensorflow.keras.callbacks import ModelCheckpoint

warnings.filterwarnings('ignore', category=UserWarning, module='keras')

checkpoint = ModelCheckpoint(
    'model_eng_weights.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    patience=10,
    verbose=1
)

In [4]:
epochs = 50

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[checkpoint],
)

Epoch 1/50
11422/18860 [=================>............] - ETA: 4:27 - loss: 1.4930 - accuracy: 0.6365

2023-11-22 01:09:03.624734: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x2910fd4e0>
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ke

11442/18860 [=================>............] - ETA: 4:26 - loss: 1.4924 - accuracy: 0.6367

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x2910fd4e0>
Traceback (most recent call last):

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Library/Frameworks/Python.framework/Vers

In [ ]:
import pandas as pd

history_df = pd.DataFrame(history.history)
history_df.to_csv('training_history_eng.csv', index=False)

history_df

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

plt.figure(figsize=(12, 6))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Train and Validation Accuracy (model_eng)')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Train and Validation Loss (model_eng)')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import classification_report

predictions = model.predict(validation_generator)

predicted_labels = predictions.argmax(axis=1)
true_labels = validation_generator.classes

class_names = list(validation_generator.class_indices.keys())

report = classification_report(true_labels, predicted_labels, target_names=class_names)
print(report)